In [4]:
# import pandas as pd
# import numpy as np
# import cupy, cudf
# import matplotlib.pyplot as plt, gc, os

ModuleNotFoundError: No module named 'cupy'

In [2]:
data_test = pd.read_csv('./data/train_data.csv')

In [3]:
data_test.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
import warnings, gc

warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)
temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), height=500, width=1000))

In [25]:
train = pd.read_feather('./data/train_data.ftr')
train.sample(5)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
2673028,7b6212c125bdeab5689682e57225e349d5b51b875372e7...,2017-07-15,0.778320,0.921387,0.117676,0.008904,0.005592,NaN,0.401123,0.046692,...,NaN,NaN,0.006973,0.008690,0.006916,NaN,0.007778,0.008652,0.006439,0
1432569,4201526deafd5917823a30d735c2baf0eaa7384fda35e7...,2017-04-22,0.953125,0.000034,0.004845,0.815430,0.009247,NaN,0.007172,0.000599,...,NaN,NaN,0.003542,0.001542,0.008354,NaN,0.001884,0.002382,0.008301,0
3036925,8c3ace6a29b8bb9fab4bd69ada6e978b41bfebeaa61367...,2018-03-19,-0.091919,0.003824,0.064453,0.007652,2.009766,0.304932,0.599121,0.104065,...,NaN,NaN,0.005936,0.007988,0.002884,NaN,0.008698,0.002531,0.000740,1
896872,2991880778c6575d6742acc55ce93d6386c7d49ce9892f...,2017-08-03,0.294189,0.647949,0.135864,0.163208,0.001027,0.445801,0.001061,0.099243,...,NaN,NaN,0.008835,0.008888,0.004883,NaN,0.006107,0.000164,0.005081,1
4369752,c9f3e824f260985aa1ce14c9af3de1dae33a4fe2ce7562...,2017-08-08,0.544434,0.000456,0.000865,0.817871,0.505859,0.161987,0.009171,0.001167,...,NaN,NaN,0.008209,0.007942,0.007797,NaN,0.004826,0.001702,0.004803,1


In [26]:
train = train.groupby('customer_ID').tail(1).set_index('customer_ID')

In [27]:
train.head()

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,2018-03-13,0.934570,0.009117,0.009384,1.007812,0.006104,0.135010,0.001604,0.007175,NaN,...,NaN,NaN,0.007187,0.004234,0.005085,NaN,0.005810,0.002970,0.008530,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,2018-03-25,0.880371,0.178101,0.034698,1.003906,0.006912,0.165527,0.005550,0.005070,NaN,...,NaN,NaN,0.002981,0.007481,0.007874,NaN,0.003284,0.003170,0.008514,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,2018-03-12,0.880859,0.009705,0.004284,0.812500,0.006451,NaN,0.003796,0.007195,NaN,...,NaN,NaN,0.007381,0.006622,0.000965,NaN,0.002201,0.000834,0.003445,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,2018-03-29,0.621582,0.001082,0.012566,1.005859,0.007828,0.287842,0.004532,0.009941,NaN,...,NaN,NaN,0.002705,0.006184,0.001899,NaN,0.008186,0.005558,0.002983,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,2018-03-30,0.872070,0.005573,0.007679,0.815918,0.001247,NaN,0.000231,0.005527,NaN,...,NaN,NaN,0.002974,0.004162,0.005764,NaN,0.008156,0.006943,0.000905,0


In [28]:
print(f"The training data begins on {train['S_2'].min().strftime('%m-%d-%Y')} and ends on {train['S_2'].max().strftime('%m-%d-%Y')}")

The training data begins on 03-01-2018 and ends on 03-31-2018


In [29]:
print(f"There are {train.shape[0]} customers in the training set and {train.shape[1]} features.")

There are 458913 customers in the training set and 190 features.


In [30]:
test = pd.read_feather('./data/test_data.ftr')

In [31]:
test.shape

(11363762, 190)

In [32]:
test = test.groupby('customer_ID').tail(1).set_index('customer_ID') # groupby后返回每组的最后一行


In [33]:
print("\nThe test data begins on {} and ends on {}.".format(test['S_2'].min().strftime('%m-%d-%Y'),test['S_2'].max().strftime('%m-%d-%Y')))
print("There are {:,.0f} customers in the test set and {} features.".format(test.shape[0],test.shape[1]))


The test data begins on 04-01-2019 and ends on 10-31-2019.
There are 924,621 customers in the test set and 189 features.


In [34]:
del test['S_2']
gc.collect()

1343

In [35]:
titles = []
for i in train.columns[:-1]:
    if i.startswith('D'):
        titles.append('Delinquency ' + str(i).split('_')[1])
    elif i.startswith('S'):
        titles.append('Spend ' + str(i).split('_')[1])
    elif i.startswith('P'):
        titles.append('Payment ' + str(i).split('_')[1])
    elif i.startswith('B'):
        titles.append('Balance ' + str(i).split('_')[1])
    elif i.startswith('R'):
        titles.append('Risk ' + str(i).split('_')[1])

In [36]:
cat_cols=['Balance 30', 'Balance 38', 'Delinquency 63', 'Delinquency 64', 'Delinquency 66', 'Delinquency 68',
          'Delinquency 114', 'Delinquency 116', 'Delinquency 117', 'Delinquency 120', 'Delinquency 126', 'Target']

In [37]:
test.columns = titles[1:]  # titles第一个市Spend2
titles.append('Target')
train.columns = titles

In [38]:
target=train.Target.value_counts(normalize=True)
target.rename(index={1:'Default',0:'Paid'},inplace=True)
pal, color=['#016CC9','#DEB078'], ['#8DBAE2','#EDD3B3']
fig=go.Figure()
fig.add_trace(go.Pie(labels=target.index, values=target*100, hole=.45, 
                     showlegend=True,sort=False, 
                     marker=dict(colors=color,line=dict(color=pal,width=2.5)),
                     hovertemplate = "%{label} Accounts: %{value:.2f}%<extra></extra>"))
fig.update_layout(template=temp, title='Target Distribution', 
                  legend=dict(traceorder='reversed',y=1.05,x=0),
                  uniformtext_minsize=15, uniformtext_mode='hide',width=700)
fig.show()

In [39]:

target=pd.DataFrame(data={'Default':train.groupby('Spend 2')['Target'].mean()*100})
target['Paid']=np.abs(train.groupby('Spend 2')['Target'].mean()-1)*100
rgb=['rgba'+str(matplotlib.colors.to_rgba(i,0.7)) for i in pal]
fig=go.Figure()
fig.add_trace(go.Bar(x=target.index, y=target.Paid, name='Paid',
                     text=target.Paid, texttemplate='%{text:.0f}%', 
                     textposition='inside',insidetextanchor="middle",
                     marker=dict(color=color[0],line=dict(color=pal[0],width=1.5)),
                     hovertemplate = "<b>%{x}</b><br>Paid accounts: %{y:.2f}%"))
fig.add_trace(go.Bar(x=target.index, y=target.Default, name='Default',
                     text=target.Default, texttemplate='%{text:.0f}%', 
                     textposition='inside',insidetextanchor="middle",
                     marker=dict(color=color[1],line=dict(color=pal[1],width=1.5)),
                     hovertemplate = "<b>%{x}</b><br>Default accounts: %{y:.2f}%"))
fig.update_layout(template=temp,title='Distribution of Default by Day', 
                  barmode='relative', yaxis_ticksuffix='%', width=1400,
                  legend=dict(orientation="h", traceorder="reversed", yanchor="bottom",y=1.1,xanchor="left", x=0))
fig.show()